# 📝 HTR using arshjot/Handwritten-Text-Recognition
## Using the actual pre-trained CRNN model from the repo

This notebook uses the **exact code** from [arshjot/Handwritten-Text-Recognition](https://github.com/arshjot/Handwritten-Text-Recognition) with their pre-trained model.

## 1️⃣ Install TensorFlow 1.15 (Required)

In [ ]:
# Install TensorFlow 1.15 (last version before 2.x)
!pip install tensorflow==1.15.0
!pip install opencv-python-headless numpy matplotlib

## 2️⃣ Clone the Repository

In [ ]:
!git clone https://github.com/arshjot/Handwritten-Text-Recognition.git
%cd Handwritten-Text-Recognition

## 3️⃣ Download Pre-trained Model

In [ ]:
# Download model from Google Drive
!pip install gdown
!gdown 1D97_MO_bOxfqxiJ8dtpbVX-xNzwQW0mY

# Extract
!tar -xzf CRNN_h128.tar.gz

# Move to correct location
!mkdir -p experiments
!mv CRNN_h128 experiments/

print("✅ Model downloaded and extracted")

## 4️⃣ Upload Your Images

In [ ]:
from google.colab import files
import os

# Create samples directory
!mkdir -p samples

print("📤 Upload your handwritten page image")
uploaded = files.upload()

# Move uploaded file to samples
for filename in uploaded.keys():
    !mv {filename} samples/
    print(f"✅ Uploaded: {filename}")

## 5️⃣ Segment Lines from Your Page

The model works on **individual line images**, so we need to segment your full page first.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

def segment_lines(image_path, output_dir='samples', min_line_height=20):
    """Segment page into lines"""
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Denoise
    denoised = cv2.fastNlMeansDenoising(gray, h=10)
    
    # Binarize
    binary = cv2.adaptiveThreshold(
        denoised, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 21, 10
    )
    
    # Horizontal projection
    h_projection = np.sum(binary, axis=1)
    threshold = np.max(h_projection) * 0.1
    
    # Find line boundaries
    in_line = False
    line_start = 0
    lines = []
    
    for i, val in enumerate(h_projection):
        if not in_line and val > threshold:
            line_start = i
            in_line = True
        elif in_line and val < threshold:
            line_end = i
            if line_end - line_start > min_line_height:
                lines.append((line_start, line_end))
            in_line = False
    
    if in_line:
        lines.append((line_start, len(h_projection)))
    
    # Save line images
    line_files = []
    for idx, (y_start, y_end) in enumerate(lines):
        # Add padding
        y_start = max(0, y_start - 5)
        y_end = min(gray.shape[0], y_end + 5)
        
        # Extract line
        line_img = gray[y_start:y_end, :]
        
        # Find horizontal boundaries
        v_projection = np.sum(binary[y_start:y_end, :], axis=0)
        non_zero = np.where(v_projection > 0)[0]
        
        if len(non_zero) > 0:
            x_start = max(0, non_zero[0] - 10)
            x_end = min(gray.shape[1], non_zero[-1] + 10)
            line_img = line_img[:, x_start:x_end]
            
            # Save
            line_file = f'{output_dir}/line_{idx:02d}.png'
            cv2.imwrite(line_file, line_img)
            line_files.append(line_file)
    
    return line_files

# Segment the uploaded image
uploaded_file = list(uploaded.keys())[0]
original_path = f'samples/{uploaded_file}'

print("🔍 Segmenting lines...")
line_files = segment_lines(original_path)
print(f"✅ Created {len(line_files)} line images")

# Visualize
for i, line_file in enumerate(line_files[:5]):
    img = cv2.imread(line_file, cv2.IMREAD_GRAYSCALE)
    plt.figure(figsize=(12, 2))
    plt.imshow(img, cmap='gray')
    plt.title(f'Line {i+1}')
    plt.axis('off')
    plt.show()

## 6️⃣ Run Prediction

In [ ]:
# Run the prediction script
%cd mains
!python predict.py -c ../configs/config.json

## 7️⃣ View Results

In [ ]:
# Results are printed above, but let's also save them
print("\n" + "="*80)
print("RECOGNITION COMPLETE")
print("="*80)
print("\nCheck the output above for recognized text from each line!")

## ⚠️ Troubleshooting

If you get TensorFlow errors:
1. **Restart runtime**: Runtime → Restart runtime
2. **Re-run from cell 1**

If recognition is poor:
- Ensure text is **black on white** background
- Try adjusting image contrast/brightness
- Check that lines are properly segmented (see visualizations above)